In [1]:
# Load Model
from pathlib import Path
import sys
sys.path.append("../")

import torch

from models.net import SoyBeanNet



MODEL_PATH = "/home/carlyn.1/dna-trait-analysis/results/feb19/all_genes_forewings_color_3/model.pt"

model = SoyBeanNet(window_size=340202, num_out_dims=10, insize=3, hidden_dim=10).cuda()
weights = torch.load(MODEL_PATH)
model.load_state_dict(weights)

<All keys matched successfully>

In [3]:
# Load Data
import numpy as np
import random
from experiments import get_all_gene_experiments
from data_tools import load_json, parse_patternize_csv
experiments = get_all_gene_experiments("erato", "forewings", "color_3")

pca_data = parse_patternize_csv(experiments[0].pca_loading_path)
for i, experiment in enumerate(experiments):
    if i == 0:
        input_data = np.load(experiment.gene_vcf_path)['arr_0']
        metadata = load_json(experiment.metadata_path)
    else:
        input_data = np.hstack((input_data, np.load(experiment.gene_vcf_path)['arr_0']))
        new_metadata = load_json(experiment.metadata_path)
        pca_data = parse_patternize_csv(experiment.pca_loading_path)
        for j, m in enumerate(metadata):
            assert m == new_metadata[j], f"Metadata does not match: {m} != {new_metadata[j]}"
        

train_data = []
print(f"Length of input data: {len(input_data)}")
for name, row in zip(metadata, input_data):
    if name+"_d" in pca_data:
        train_data.append([name, row, pca_data[name+"_d"]])
print(f"Length of train data: {len(train_data)}")

random.seed(2)
random.shuffle(train_data)
train_idx = int(len(train_data) * 0.8)
val_idx = int(len(train_data) * 0.1)

train_split = train_data[:train_idx]
val_split = train_data[train_idx:train_idx+val_idx]
test_split = train_data[train_idx+val_idx:]


Length of input data: 484
Length of train data: 480


In [4]:
from captum.attr import Occlusion

def get_attribution_points(model, dloader):
    att_model = Occlusion(model)
    attr_total = None
    for i, batch in enumerate(dloader):
        model.zero_grad()
        name, data, pca = batch
        attr = att_model.attribute(data.cuda(), target=0, sliding_window_shapes=(1, 200, 3), strides=20, show_progress=True)
        #attr = att_m.attribute(data.cuda(), target=0, show_progress=True)
        #attr = attr.abs() # Just take the abs value
        #attr, _ = attr.max(-1) # Max across 1-hot representation of input
        attr = attr.sum(-1)
        attr = attr[:, 0] # Only has 1 channel, just extract it
        attr = attr.sum(0) # Sum across batch
        if attr_total is None:
            attr_total = attr.detach().cpu().numpy()
        else:
            attr_total += attr.detach().cpu().numpy()

    #attr_total = attr_total / np.linalg.norm(attr_total, ord=1) # Normalize
    return attr_total

In [5]:
from torch.utils.data import DataLoader
from data_tools import VCF_Dataset

dl = DataLoader(VCF_Dataset(test_split), batch_size=256, num_workers=8, shuffle=False)
attr_total = get_attribution_points(model, dl)

Occlusion attribution:   0%|          | 0/17003 [00:00<?, ?it/s]

/home/carlyn.1/miniconda3/envs/dna/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1699449201450/work/aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv2d(input, weight, bias, self.stride,
